#Batch normalization Step by Step

- 신경망이 깊어질 수록 은닉층(hidden layer)은 학습이 어려워진다.
- Batch normalization 논문의 저자는 이를 Internal Covariate Shift라고 한다.

-인공 신경망에서의 Internal Covariate Shift란?
- 지속적인 weight update로 인해 Hidden layer의 input의 분포가 바뀌기때문에, 모델의 학습이 어려워 지는 현상을 말한다.
- __인공 신경망의 각 층은 이전 층의 분포를 학습하기 때문이다.__
- *Batch Normalization: Accelerating Deep Network Training by reducing Internal Covariate Shift*에서 Internal covariate shift를 해결하기 위해 배치 단위의 정규화를 제안하였다.

 -Mini batch의 정규화란?
 - 올바른 정규화 방법은 모든 데이터셋의 평균과 분산을 구해서 정규화 해줘야한다.
 
 
$$
\hat X^{(k)} = \frac{x^{(k)}-E[x^{(k)}]}{\sqrt{Var[x^{(k)}]}}
$$
 
 - 그러나 이는 너무 만은 계산비용을 발생하므로, __Mini batch__별로 분산과 평균을 구해 빠르게 정규화한다.
  
  
$$
\mu_{B}^{(k)} = \frac{1}{m}\sum_{i=1}^{m}x_i^{(k)} $$
<br> $$
(\sigma_{B}^{(k)})^2 = \frac{1}{m}\sum_{i=1}^{m}(x_i^{(k)}-\mu_B^{(k)})^2 $$ <br>
$$ \hat x^{(k)} = \frac{x_i - \mu_{B}}{\sqrt{\sigma_{B}^2+\epsilon}} $$


- Scale factor / Shift factor <br>
그러나 모든 층에서의 평균은 0이고 분산은 1인 정규분포 형태를 띄게 될경우, <br>
각 층마다 적절한 수준의 평균과 분산과는 다를 수 있기 때문에 학습에 지장을 초래한다.<br>
따라서 loss에 대해 학습으로 update되는 variable 2개를 추가하여 정규분포를 scaling한다. <br> 
즉, 2개의 Weight, Scale Factor( γ(k) )와 Shift Factor( β(k) )를 별도로 구성한다. <br>
$y^{(k)} = \gamma^{(k)}\hat x ^{(k)} + \beta^{(k)}$

##-최종 배치정규화의 진행 순서 <Br>
1.$
z = X\cdot W, \cdots \mbox{ (1) 로짓 계산}$<br>
2.$y = BN(z), \cdots \mbox{ (2) 배치노말 적용 }$ <br>
3.$a = \sigma(y), \cdots \mbox{ (3) 활성화 함수 적용 }$<br>

<br>
위와 같은 순서로 batch normalization을 수행한다.

In [0]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import tensorflow as tf
import cv2


평균과 분산을 구하는 메소드로 Tensorflow에서 `tf.nn.moments`를 사용<br>


```python
graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,(None,32),name='inputs')
    epsilon = 1e-3
    with tf.variable_scope('normalization'):
        batch_mean, batch_var = tf.nn.moments(x,[0])
        x_norm = (x-batch_mean)/tf.sqrt(batch_var+epsilon)
```


Scale Factor($\gamma$)와 Shift Factor($\beta$) 구성

Scale Factor은 1로, Shift Factor는 0으로 초기화

```python
with graph.as_default():
    input_size = x.get_shape()[-1] #마지막 -1
    
    # featuare갯수만큼 scale factor와 shift factor
    gamma = tf.Variable(tf.ones(input_size), name='scale_factor')
    beta = tf.Variable(tf.zeros(input_size), name='shift_factor')
    
    with tf.variable_scope('transformation'):
        y = gamma * x_norm + beta #elememet wise
```

Test때 사용할 분산과 평균을 구하기위해 Moving average(지수평균이동)를 사용한다.

결론적으로 Batch normalizatoin의 test/ train시의 코드는 다음과 같다.

```python
import tensorflow as tf
decay = 0.999 #beta
graph = tf.Graph()
eps = 0.0000001

with graph.as_default():
    xs = tf.placeholder(dtype = tf.float32, shape = [None,784])
    phase_train =  tf.placeholder(dtype = tf.bool, shape = [])
    sizes = xs.get_shape()[-1]
    
    #moving average의 초기값 0
    test_mean = tf.Variable(tf.zeros([sizes]), name = 'test_mean')
    test_var = tf.Variable(tf.zeros([sizes]), name = 'test_var')
    
    # featuare갯수만큼 scale factor와 shiftf factor
    gamma = tf.Variable(tf.ones(input_size), name='scale_factor')
    beta = tf.Variable(tf.zeros(input_size), name='shift_factor')

    def train():
        mean, var = tf.nn.moments(xs, [0])
        #이동평균 mean/var
        updated_mean = (((1-decay) * mean) + (decay*(test_mean)))
        updated_var = (((1-decay) * var) + (decay*(test_var)))   
        
        with tf.control_dependencies([updated_mean ,updated_var]) :
            xs_norm = (xs - mean) /tf.sqrt(var+eps)
            xs_bn = (xs_norm * gamma) + beta
            return xs_bn    
    
    def test():
        #test할때나 train할때나 betad와 gamma같다
        xs_norm = (xs - test_mean) / tf.sqrt(test_var+eps)
        xs_bn = (xs_norm*gamma) + beta
        return xs_bn
        
    tf.cond(phase_train, train, test)
```

- tf.nn.batch_normalizatoin을 사용하여 wrapper method로 정리하여 사용한다.

In [0]:
def batch_normalization(inputs, is_train, decay=0.999, epsilon=1e-5):    
    #input shape의 feature size측정
    sizes = inputs.get_shape()[-1]
    
    #test를 위한 moving average를 계산위함, 초기값은 0으로 지정 : tf.zeros
    test_mean = tf.Variable(tf.zeros([sizes]), name = 'test_mean')
    test_var = tf.Variable(tf.zeros([sizes]), name = 'test_var')
    
    # featuare갯수만큼 scale factor와 shiftf factor
    gamma = tf.Variable(tf.ones(input_size), name='scale_factor')
    beta = tf.Variable(tf.zeros(input_size), name='shift_factor')

    def train():
        mean, var = tf.nn.moments(inputs, [0])
        updated_mean = tf.assign(test_mean, 
                                 (((1-decay) * mean) + (decay*(test_mean))))
        updated_var = tf.assign(test_var,
                                (((1-decay) * var) + (decay*(test_var))))
        
        with tf.control_dependencies([updated_mean ,updated_var]) :
            xs_bn = tf.nn.batch_normalization(inputs, mean, var, 
                                              beta, gamma, eps)
            
            #xs_norm = (xs - mean) /tf.sqrt(var+eps)
            #xs_bn = (xs_norm * gamma) + beta
            
            return xs_bn    
    
    def test():
        #test할때나 train할때나 beta gamma는 동일함
        #xs_norm = (xs - test_mean) / tf.sqrt(test_var+eps)
        #xs_bn = (xs_norm*gamma) + beta
        #high api사용
        xs_bn = tf.nn.batch_normalization(inputs, mean, var, beta, gamma, eps)
        
        return xs_bn
        
    xs_bn = tf.cond(phase_train, train, test)
    
    return xs_bn